In [6]:
import os
import networkx as nx
import torch
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data

In [7]:
def generate_structural_anomaly(num_normal=250, seed=42):
    torch.manual_seed(seed)

    
    G_normal = nx.barabasi_albert_graph(n=num_normal, m=3, seed=seed)

    
    G_clique = nx.complete_graph(5)  
    G_path = nx.path_graph(6)        
    G_star = nx.star_graph(5)        

    def relabel(g, offset):
        return nx.relabel_nodes(g, lambda x: x + offset)

    G_clique = relabel(G_clique, num_normal)
    G_path = relabel(G_path, num_normal + 5)
    G_star = relabel(G_star, num_normal + 11)

    
    G_all = nx.disjoint_union_all([G_normal, G_clique, G_path, G_star])
    num_nodes = G_all.number_of_nodes()

    
    anomaly_nodes = list(range(num_normal, num_nodes))
    for i in range(num_nodes):
        G_all.nodes[i]['x'] = [G_all.degree[i]]
        G_all.nodes[i]['y'] = int(i in anomaly_nodes)

    
    data = from_networkx(G_all)
    data.x = torch.tensor([data.x[i] for i in range(data.num_nodes)], dtype=torch.float)
    data.y = torch.tensor([data.y[i] for i in range(data.num_nodes)], dtype=torch.long)

    return data

In [8]:

data = generate_structural_anomaly()
save_path = "/data1/shengen/STATS403/project/data/structural_anomaly_graph.pt"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(data, save_path)
save_path

'/data1/shengen/STATS403/project/data/structural_anomaly_graph.pt'